In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"E:\courses\natural lang processing\data sets for coure 1\fake_news_train.csv")

In [4]:
df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df = df.dropna()

In [6]:
df.reset_index(inplace = True)

In [7]:
df.head()
df.shape
x = df.drop("label" , axis = 1)


In [8]:
y = df["label"]


In [9]:
#data preprocessing

import re
import nltk
from nltk.corpus import stopwords

corpus = []

from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

for i in range(0, len(x)):
    review = re.sub('[^a-zA-Z]', ' ', x['title'][i])
    review = review.lower()
    review = review.split()
    review = [lem.lemmatize(word) for word in review if word not in (stopwords.words("english"))]
    review = " ".join(review)
    
    corpus.append(review)


In [10]:
# implementing one hot representation

from tensorflow.keras.preprocessing.text import one_hot

In [11]:
vocab = 5000

onehot_repr = [one_hot(word , vocab ) for word in corpus]

In [12]:
onehot_repr[0]

[3145, 4526, 3022, 1405, 2285, 777, 3816, 3809, 2251, 3372]

In [13]:
#padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
sent_max_len = 20

embedding_doc = pad_sequences(onehot_repr , padding = "pre" , maxlen = sent_max_len )

In [15]:
embedding_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3145,
       4526, 3022, 1405, 2285,  777, 3816, 3809, 2251, 3372])

In [16]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Embedding , Dense , LSTM




In [17]:
#creating model

feature_vector = 50

sq = Sequential()
sq.add(Embedding(vocab , feature_vector , input_length = sent_max_len ))
sq.add(LSTM(100))
sq.add(Dense(1 , activation = "sigmoid"))

# added all the layers, now compiling this model

sq.compile(loss = "binary_crossentropy", optimizer = 'adam' , metrics = ["accuracy"])

sq.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [18]:
import numpy as np


In [19]:
X = np.array(embedding_doc)
Y =np.array(y) 

In [20]:
# creating train-test-split and training the model

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y , test_size = 0.33 , random_state = 42)

In [21]:
X.shape , y.shape

((18285, 20), (18285,))

In [22]:
#training

sq.fit(xtrain , ytrain, validation_data = (xtest , ytest), epochs = 10 , batch_size = 64)

Epoch 1/10
192/192 [==============================] - 10s 33ms/step - loss: 0.3239 - accuracy: 0.8496 - val_loss: 0.1947 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1358 - accuracy: 0.9463 - val_loss: 0.2137 - val_accuracy: 0.9180
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0920 - accuracy: 0.9660 - val_loss: 0.2234 - val_accuracy: 0.9109
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0630 - accuracy: 0.9776 - val_loss: 0.2890 - val_accuracy: 0.9095
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0384 - accuracy: 0.9869 - val_loss: 0.3604 - val_accuracy: 0.9102
Epoch 6/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0230 - accuracy: 0.9941 - val_loss: 0.3985 - val_accuracy: 0.9114
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0147 - accuracy: 0.9958 - val_loss: 0.4887 - val_accuracy: 0.904

In [29]:
# performance metrics and accuracy

pred = (sq.predict(xtest) > 0.5).astype("int32")


from sklearn.metrics import confusion_matrix , accuracy_score

confusion_matrix(pred, ytest), accuracy_score(pred, ytest)

(array([[3123,  268],
        [ 296, 2348]], dtype=int64),
 0.9065451532725767)